In [7]:
import json
from tqdm import tqdm_notebook as tqdm
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
import os

In [8]:
with open('station_location.json', 'r') as infile:
    station_location = json.load(infile)

In [15]:
station_location.keys()

dict_keys(['0', '3', '40', '18', '41', '48', '27', '14', '11', '31', '1', '36', '37', '49', '20', '9', '5', '30', '43', '32', '12', '24', '8', '2', '10', '22', '13', '39', '38', '23', '25', '28', '46', '26', '33', '7', '15', '21', '35', '29', '45', '47', '4', '6', '42', '19', '44', '17', '34', '16'])

In [145]:
filepath = './datasets'
gauge_path = filepath+'/gauge.csv'
radar_path = filepath+'/radar'

In [146]:
df_gauge = pd.read_csv(gauge_path)

In [147]:
df_gauge_array = df_gauge.values[:, 2:]
print (df_gauge_array.shape)

(52, 50)


In [161]:
np.where(pd.isnull(df_gauge))

(array([], dtype=int64), array([], dtype=int64))

In [148]:
all_radar_path_list = sorted(os.listdir(radar_path))

In [149]:
df_gauge.head()
df_gauge['Week'] = df_gauge['Week'].astype(str)

In [150]:
train_week = []
list_dfs = []
for filename in tqdm(all_radar_path_list):
    week = filename.split('.')[0][-2:]
    train_week.append(str(week))
    full_radar_path = os.path.join(radar_path, filename)
    df_radar = pd.read_csv(full_radar_path)
    df_radar['latlong'] = df_radar['lat'] * 1000 + df_radar['long']
    list_dfs.append(df_radar)

HBox(children=(IntProgress(value=0, max=31), HTML(value='')))

In [159]:
df_result = pd.DataFrame(columns = column_names)
for station in tqdm(range(df_gauge_array.shape[1])):
    for week in train_week:
        chosen_latlong = int(station_location[str(station)]['latlong'])
        y = df_gauge[df_gauge['Week'] == str(week)][str(station)].values[0]
        week_index = train_week.index(week)
        df_radar = list_dfs[week_index]
        x = df_radar[df_radar['latlong'] == chosen_latlong].values[:, 2:36][0]
        df_row = pd.DataFrame([np.concatenate(([station], [week], x, [y]))], columns = column_names)
        df_result = pd.concat([df_result, df_row])      

HBox(children=(IntProgress(value=0, max=50), HTML(value='')))

In [163]:
df_result.to_csv('train.csv')